## Notebook responsável por gerar o dataset de entrada para o mapa

In [ ]:
import base64
import math
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests as req

### Carrega datasets 

In [ ]:
# df de acidentes mais atualizado e pré processado
df_acidentes = pd.read_excel("../dados/modificados/BD Acidentes_01_07_processado.xlsx")
# df de todas as OS, com Local de instalação e Trabalho Real
dfOS = pd.read_excel("../dados/6-ESUL-IW49-LINHA-Ordens e Operações-PM-set23.xlsx")
# criacao do df de local/coordenada a partir das novas tabelas
df_coord_instalacoes = pd.read_excel("../dados/ESUL-LIs-exceto linhas.xlsx")
# reduzindo as colunas que não serão utilizadas
df_coord_instalacoes = df_coord_instalacoes[
    ["Local de instalação", "Latitude", "Longitude"]
]

In [ ]:
# cria um dicionario que consegue ser utilizado em um .replace() para conseguir
# agrupar as siglas de subestação checando se a string da sigla pertence a string de outra sigla na coluna


def agrupa_siglas_de_instalacao(dfLocais, colSigla, lista_siglas=None):

    dictTraducoes = {}

    if lista_siglas == None:
        lista_siglas = dfLocais[colSigla].dropna(inplace=True)
        lista_siglas = dfLocais[colSigla].unique()

    para_substituir = dfLocais[colSigla].unique()
    for sigla in lista_siglas:
        for subconjunto in para_substituir:
            if sigla in subconjunto:
                if subconjunto not in dictTraducoes:
                    dictTraducoes[subconjunto] = sigla
                else:
                    if sigla in dictTraducoes[subconjunto]:
                        dictTraducoes[subconjunto] = sigla
    return dictTraducoes

In [ ]:
df_siglas_importantes = df_coord_instalacoes.dropna(subset=["Latitude", "Longitude"])

dict_traducao1 = agrupa_siglas_de_instalacao(
    df_siglas_importantes, "Local de instalação"
)


# criando a lista com as siglas dos locais com coordenadas
lista_siglas_boas = {}
for sigla in dict_traducao1.values():
    lista_siglas_boas[sigla] = 1
# é necessário este passo senão todas as siglas viram
# a menor possível, que é apenas 'S', que inclusive não tem coordenada associada

# com este dict de tradução podemos agora substituir nas OS para que todas tenham alguma coordenada associada
dict_traducao = agrupa_siglas_de_instalacao(
    df_coord_instalacoes, "Local de instalação", lista_siglas=lista_siglas_boas
)

In [ ]:
# função que elimina repetições de siglas em um df


def cria_df_mapa(dfLocalComScore, colSigla):

    # reconstroi o dfLocalComScore agrupando as linhas que usam a mesma sigla

    df = pd.DataFrame()

    df[colSigla] = dfLocalComScore[colSigla].dropna().unique()
    try:
        for i, row in df.iterrows():
            sigla = df.loc[i, colSigla]
            idx = dfLocalComScore.index[dfLocalComScore[colSigla] == sigla].to_list()
            df.loc[i, "Latitude"] = dfLocalComScore.loc[idx[0], "Latitude"]
            df.loc[i, "Longitude"] = dfLocalComScore.loc[idx[0], "Longitude"]
            df.loc[i, "horas de OS"] = dfLocalComScore.loc[idx, "horas de OS"].sum()
            df.loc[i, "qtd de OS"] = dfLocalComScore.loc[idx, "qtd de OS"].sum()
            df.loc[i, "peso acidentes"] = dfLocalComScore.loc[
                idx, "peso acidentes"
            ].sum()
            df.loc[i, "qtd de acidentes"] = dfLocalComScore.loc[
                idx, "qtd de acidentes"
            ].sum()
            df.loc[i, "qtd de acidentes alto potencial"] = dfLocalComScore.loc[
                idx, "qtd de acidentes alto potencial"
            ].sum()
            if df.loc[i, "horas de OS"] == 0:
                df.loc[i, "Score"] = np.nan
            else:
                df.loc[i, "Score"] = (
                    df.loc[i, "peso acidentes"] / df.loc[i, "horas de OS"]
                )
        df = df[df["horas de OS"] != 0]
    except KeyError:
        for i, row in df.iterrows():
            sigla = df.loc[i, colSigla]
            idx = dfLocalComScore.index[dfLocalComScore[colSigla] == sigla].to_list()
            df.loc[i, "Latitude"] = dfLocalComScore.loc[idx[0], "Latitude"]
            df.loc[i, "Longitude"] = dfLocalComScore.loc[idx[0], "Longitude"]

    return df

In [ ]:
# preprocessando as coordenadas deste df
df_coord_instalacoes["Latitude"] = (
    df_coord_instalacoes["Latitude"].str.replace(",", ".").str.replace("°", "")
)
df_coord_instalacoes["Longitude"] = (
    df_coord_instalacoes["Longitude"].str.replace(",", ".").str.replace("°", "")
)
df_coord_instalacoes["Latitude"] = df_coord_instalacoes["Latitude"].astype(float)
df_coord_instalacoes["Longitude"] = df_coord_instalacoes["Longitude"].astype(float)

In [ ]:
# ajuste do nome de variável, bacalhau a se remover depois
dfCoordenadasSubestacoesOS = df_coord_instalacoes.copy()

dfCoordenadasSubestacoesOS.dropna(subset=["Latitude", "Longitude"], inplace=True)
dfCoordenadasSubestacoesOS.replace(dict_traducao1, inplace=True)
dfCoordenadasSubestacoesOS = cria_df_mapa(
    dfCoordenadasSubestacoesOS, "Local de instalação"
)

In [ ]:
# preprocessando o dfOS para uso no calculo do indicador de risco
dfOS = dfOS[
    [
        "Local de instalação",
        "Trabalho real",
        "Trabalho previsto",
        "1ª data de início",
        "1ª data fim",
    ]
]

In [ ]:
# esta operação **DEMORA** por isso está separada sozinha em uma célula; ainda é preprocessamento
dfOS.replace(dict_traducao, inplace=True)

### Funções para poder associar um Local de instalação a um acidente

In [ ]:
# calcula distancia entre coordenadas, em km


def haversine(lat1, lon1, lat2, lon2):
    import math

    R = 6371

    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2) * math.sin(dlat / 2) + math.cos(
        math.radians(lat1)
    ) * math.cos(math.radians(lat2)) * math.sin(dlon / 2) * math.sin(dlon / 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = R * c

    return d


# adiciona duas colunas ao df de acidentes, Local de instalação e Distancia


def CalculaLocalizacaoAcidente(raio, dfAcidentes, dfInsalacaoCoordenadas):
    latLong = dfAcidentes[["Latitude", "Longitude"]].dropna()
    latLong["Longitude"] = latLong["Longitude"].apply(
        lambda x: x / 100000 if x < -100 else x
    )
    latLong["Longitude"] = latLong["Longitude"].apply(
        lambda x: x * (-1) if x > 0 else x
    )

    latLong["Longitude"] = latLong["Longitude"].apply(
        lambda x: x * 10 if x > -10 else x
    )
    latLong = latLong[latLong["Latitude"] > -40]

    for index, row in latLong.iterrows():
        menor_distancia = raio
        idx_menor_distancia = -1
        for index2, row2 in dfInsalacaoCoordenadas.iterrows():
            distance = haversine(
                row2["Latitude"], row2["Longitude"], row["Latitude"], row["Longitude"]
            )
            if distance < menor_distancia:
                menor_distancia = distance
                idx_menor_distancia = index2
        if idx_menor_distancia == -1:
            continue
        dfAcidentes.loc[index, "Local de instalação"] = dfInsalacaoCoordenadas.loc[
            idx_menor_distancia, "Local de instalação"
        ]
        # colocar coluna distancia para a subestação mais proxima
        dfAcidentes.loc[index, "Distancia"] = menor_distancia

    return dfAcidentes

In [ ]:
# adicionando ao df_acidentes as colunas necessárias
raio = 50
df_acidentes = CalculaLocalizacaoAcidente(
    raio, df_acidentes, dfCoordenadasSubestacoesOS
)

### Funções para calcular o Indicador de risco

In [ ]:
# função que conta as horas trabalhadas e a quantidade de OS que aconteceram em um local específico


def conta_OS_local(df, nome_do_lugar, data_inicio=None, data_final=None):

    OS_no_local = df.loc[df["Local de instalação"] == nome_do_lugar]
    # | (df['Local de instalação'] == nome_do_lugar)

    if OS_no_local.shape[0] == 0:
        return np.nan, np.nan

    if data_inicio is None or data_final is None:
        horas_trabalhadas = OS_no_local["Trabalho real"].sum()
        trabalho_real_zero = OS_no_local.loc[OS_no_local["Trabalho real"] == 0]
        horas_trabalhadas += trabalho_real_zero["Trabalho previsto"].sum()
        return OS_no_local.shape[0], horas_trabalhadas

    OS_entre_dias = OS_no_local[
        (OS_no_local["1ª data de início"] <= pd.to_datetime(data_final))
        & (OS_no_local["1ª data fim"] >= pd.to_datetime(data_inicio))
    ]

    horas_trabalhadas = OS_entre_dias["Trabalho real"].sum()
    trabalho_real_zero = OS_entre_dias.loc[OS_entre_dias["Trabalho real"] == 0]
    horas_trabalhadas += trabalho_real_zero["Trabalho previsto"].sum()

    return OS_entre_dias.shape[0], horas_trabalhadas


# função principal, mudar o nome para ficar de acordo


def score_novo_com_local_em_acidente(dfCoordenadas, df_acidente_com_local):
    for i, linha in dfCoordenadas.iterrows():

        acidentes_no_local = df_acidente_com_local[
            df_acidente_com_local["Local de instalação"] == linha["Local de instalação"]
        ]

        qtdOSnaSubstacao, horasNaSubstacao = conta_OS_local(
            dfOS, linha["Local de instalação"]
        )
        dfCoordenadas.loc[i, "qtd de OS"] = qtdOSnaSubstacao
        dfCoordenadas.loc[i, "horas de OS"] = horasNaSubstacao
        qtdAcidentes = acidentes_no_local.shape[0]
        qtdAltoPotencial = acidentes_no_local[acidentes_no_local["Potencial"] <= 8]
        dfCoordenadas.loc[i, "qtd de acidentes"] = qtdAcidentes
        dfCoordenadas.loc[i, "qtd de acidentes alto potencial"] = qtdAltoPotencial
        acidentes_no_local["peso acidentes"] = 1 / acidentes_no_local["Potencial"]

        peso_acidentes = acidentes_no_local["peso acidentes"].sum()
        dfCoordenadas.loc[i, "peso acidentes"] = peso_acidentes
        dfCoordenadas.loc[i, "Score"] = peso_acidentes / horasNaSubstacao
        if np.isnan(horasNaSubstacao):
            dfCoordenadas.loc[i, "Score"] = pd.NA
    dfCoordenadas.dropna(subset=["Score"], inplace=True)
    return dfCoordenadas


# função para normalizar o indicador de risco dentro do df


def normaliza_score(df, colScore):
    df[colScore] = df[colScore].dropna()
    df["Score Normalizado"] = 100 * (df[colScore]) / (df[colScore].max(axis=0))
    return df

In [ ]:
# calcula o indicador de risco para todas as localizações
dfCoordenadasSubestacoesOS = score_novo_com_local_em_acidente(
    dfCoordenadasSubestacoesOS, df_acidentes
)

In [ ]:
# cria o df sem repetição que será utilizado para posicionar os pontos no mapa e salva o arquivo
dfMapa = cria_df_mapa(dfCoordenadasSubestacoesOS, "Local de instalação")

dfMapa = normaliza_score(dfMapa, "Score")

### Salva o Dataset que é utilizado pelo módulo 'gera_mapa.ipynb' no folder 'modificados' 

In [ ]:
path_dataset_para_mapa = "../dados/modificados/df_mapa.xlsx"
dfMapa.to_excel(path_dataset_para_mapa, index=False)